# Check MSNBC Data

In [2]:
with open("msnbc_0004.conll") as f:
    conll_data = f.readlines()

In [17]:
with open("msnbc_formatted.conll") as f:
    formatted_data = f.readlines()

In [4]:
def check_diff(iline, line, gold_line, index, index_name):
    if line[index] != gold_line[index]:
        print(f"Line {iline}: Difference in {index_name}: {line[index]} should be {gold_line[index]}")

In [5]:
diff = 0
found_diff = False
for iline in range(4500):
    if "#begin" in conll_data[iline] or "#end" in conll_data[iline] or len(formatted_data[iline-diff].split()) == 0:
        found_diff = False
        continue
    if len(conll_data[iline].split()) == 0:
        if len(formatted_data[iline-diff].split()) != 0:
            diff = diff + 1
            found_diff = True
        continue

    if '@' in conll_data[iline].split()[3]:
        diff = diff - 1
    
    gold_line = conll_data[iline].split()
    line = formatted_data[iline - diff].split()
    try:
        #check_diff(iline, line, gold_line, 0, "title")
        #check_diff(iline, line, gold_line, 1, "part_no")
        #if not found_diff:
        #    check_diff(iline, line, gold_line, 2, "word_no")
        #check_diff(iline, [word.lower() for word in line], [word.lower() for word in gold_line], 3, "word") # not counting capitalization as an issue
        check_diff(iline, line, gold_line, 4, "POS-tag")
        #check_diff(iline, line, gold_line, 5, "parse bit")
        check_diff(iline, line, gold_line, 10, "NER label")
    
    except:
        print(f"Error on line {iline}")

Line 1: Difference in POS-tag: NN should be DT
Line 2: Difference in POS-tag: VBD should be NNS
Line 3: Difference in POS-tag: JJ should be WDT
Line 4: Difference in POS-tag: RB should be VBP
Line 5: Difference in POS-tag: TO should be JJ
Line 6: Difference in POS-tag: VB should be MD
Line 7: Difference in POS-tag: . should be VB
Error on line 9
Line 10: Difference in POS-tag: 000 should be CD
Error on line 10
Line 11: Difference in POS-tag: RB should be CD
Line 11: Difference in NER label: * should be *)
Line 12: Difference in POS-tag: NN should be NNS
Line 13: Difference in POS-tag: VBD should be .
Line 15: Difference in POS-tag: JJ should be DT
Line 16: Difference in POS-tag: RB should be NN
Line 17: Difference in POS-tag: VBN should be EX
Line 18: Difference in POS-tag: . should be VBD
Line 20: Difference in POS-tag: NN should be IN
Line 21: Difference in POS-tag: VBD should be DT
Line 22: Difference in POS-tag: RB should be NN
Line 22: Difference in NER label: * should be (PRODUCT

Note that there are hyphens on lines 830, 871, 1067, 1127, 2566, ... which are counted as sentence delimiters by the original conll format. The nltk.sent_tokenizer does not account for this, but I am not focusing on finding a better tokenizer for now, since that won't be an issue with the FAA data

### check parse bit internal consistency

pivotting to see if the parentheses line up at least

If the only outputted line is the last one in the doc, then all the parantheses get closed properly

In [18]:
depth = 0
start = False

print("  Line  |  Depth  ")
print("------------------")

for iline, line in enumerate(formatted_data):
    
    if "#begin" in line:
        start=True
        continue
    if "#end" in line:
        continue

    if len(line.split()) == 0:
        if depth != 0:
            print(f"{iline:8}|{depth:8}")
            depth = 0
        continue

    if start:
        if '(TOP' not in line:
            print(f"{iline:8}| Missed TOP")
        start = False

    parse_bit = line.split()[5]

    depth = depth + parse_bit.count('(')
    depth = depth - parse_bit.count(')')

  Line  |  Depth  
------------------
   12091| Missed TOP
   63680|       5


# Check FAA Data

In [47]:
with open("faa.conll") as f:
    faa_data = f.readlines()

In [48]:
depth = 0
start = False

print("  Line  |  Depth  ")
print("------------------")

for iline, line in enumerate(faa_data):

    if "#begin" in line:
        start=True
        continue
    if "#end" in line:
        continue

    if len(line.split()) == 0:
        if depth != 0:
            print(f"{iline:8}|{depth:8}")
            depth = 0
        continue

    if start:
        if '(TOP' not in line:
            print(f"{iline:8}| Missed TOP")
        start = False

    parse_bit = line.split()[5]

    depth = depth + parse_bit.count('(')
    depth = depth - parse_bit.count(')')

  Line  |  Depth  
------------------


In [49]:
for line in faa_data:
    if "#begin" not in line and "#end" not in line and len(line.split()) != 0 and len(line.split()) < 12:
        print(line)